# NEPOOL IDR Drop

Batches tasks for NEPOOL IDR Drop.

In [1]:
import os
basepath = '/Volumes/Samsung_T5/la-tools-test/IDR_Drop'
#basepath = os.getcwd()
os.chdir(basepath)
print('working in directory {}.'.format(basepath))

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from pandas.io.json import json_normalize
import datetime as dt
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
#import emailscrape
import json
import math
import time
import ast

working in directory /Volumes/Samsung_T5/la-tools-test/IDR_Drop.


## Read Excel file

In [2]:
loginpath = os.path.join(basepath, 'Logins')
os.chdir(loginpath)

output_df = EPOwebscrape.read_logins('email_bodies_10_11_2019.xlsx')
output_df

,accts,date,name,user,pw,util
0,"[51640724076_352382007, 51733934061_347092002,...",2019-10-11 11:36:04,Hartford Healthcare Corporation,harth-engie,78cck,CLP
1,[54202385001_052931000],2019-10-11 11:34:00,Protective Armored Systems,usar-engie,m372b,WMECO
2,"[51640724076_352382007, 51733934061_347092002,...",2019-10-11 10:34:43,Hartford Healthcare Corporation,harth-engie,78cck,CLP
3,"[54977381078_126733008, 54480402023_526541004,...",2019-10-10 15:55:06,Berkshire Medical Center Inc.,berkmc-engie,d823k,WMECO
4,[51436492052_684002003],2019-10-10 15:54:30,Teds Food Center,teds-engie,tc836,CLP
5,"[51549893006_482562007, 51874553092_860042003]",2019-10-10 09:43:48,PM Engineered Solutions,pme-engie,p899g,CLP
6,[51482323003_190712004],2019-10-09 13:12:03,Bimba Bakeries,bbak-engie,9567c,CLP


## Download files from EPO portal

In [3]:
results = []
files = []
beg = time.time()

for row in range(0, len(output_df.accts)):
    try:
        file = EPOwebscrape.idr_download(row, output_df)
        files.append(file)
        results.append(True)
        print('done with {} of {}.'.format(row+1, len(output_df.accts)))
        print('---------------------------')
    except:
        print('error, accts {}.'.format(output_df.accts[row]))
        files.append(None)
        results.append(False)
        print('done with {} of {}.'.format(row+1, len(output_df.accts)))
        print('---------------------------')

end = time.time()
elapse = round(end - beg, 2)
print('downloaded {} files in {} seconds.'.format(len(output_df.accts), elapse))
output_df['files'] = files
output_df['success'] = results

user:  harth-engie
password:  78cck
set to last  400  days.
logging on...
looking for 4 accts.
found items in portal.
trying search & download...

submitting...
downloaded EPO data file eversource_3301aae8-ccc5-444f-b3cc-59871f336398_b0946bc7_hourlycsv.csv.
done with 1 of 7.
---------------------------
user:  usar-engie
password:  m372b
set to last  400  days.
logging on...
looking for 1 accts.
found items in portal.
trying search & download...

submitting...
downloaded EPO data file eversource_ea7fc589-6e46-4e50-9311-edcf240ef6eb_ca2a3e3d_hourlycsv.csv.
done with 2 of 7.
---------------------------
user:  harth-engie
password:  78cck
set to last  400  days.
logging on...
looking for 4 accts.
found items in portal.
trying search & download...

submitting...
downloaded EPO data file eversource_6a4bf09e-05d0-4650-87d0-e5fc3a8cf588_f174016b_hourlycsv.csv.
done with 3 of 7.
---------------------------
user:  berkmc-engie
password:  d823k
set to last  400  days.
logging on...
looking for 4 

In [12]:
output_df.to_excel('output_df_10_14_2019.xlsx')
output_df

,accts,date,name,user,pw,util,files,success
0,"['51640724076_352382007', '51733934061_3470920...",2019-10-11 11:36:04,Hartford Healthcare Corporation,harth-engie,78cck,CLP,eversource_3301aae8-ccc5-444f-b3cc-59871f33639...,True
1,['54202385001_052931000'],2019-10-11 11:34:00,Protective Armored Systems,usar-engie,m372b,WMECO,eversource_ea7fc589-6e46-4e50-9311-edcf240ef6e...,True
2,"['51640724076_352382007', '51733934061_3470920...",2019-10-11 10:34:43,Hartford Healthcare Corporation,harth-engie,78cck,CLP,eversource_6a4bf09e-05d0-4650-87d0-e5fc3a8cf58...,True
3,"['54977381078_126733008', '54480402023_5265410...",2019-10-10 15:55:06,Berkshire Medical Center Inc.,berkmc-engie,d823k,WMECO,eversource_f77d133b-f6a9-4a30-bd74-6279ceb1b22...,True
4,['51436492052_684002003'],2019-10-10 15:54:30,Teds Food Center,teds-engie,tc836,CLP,eversource_e8ccb470-8a69-4927-88ba-8b90a77c250...,True
5,"['51549893006_482562007', '51874553092_8600420...",2019-10-10 09:43:48,PM Engineered Solutions,pme-engie,p899g,CLP,eversource_3cdbbc1e-68ba-48ea-8cbe-b4a77d15d30...,True
6,['51482323003_190712004'],2019-10-09 13:12:03,Bimba Bakeries,bbak-engie,9567c,CLP,eversource_368b06a1-eb32-4ab6-a29b-14d1282156c...,True


In [3]:
output_df = pd.read_excel('output_df_10_14_2019.xlsx')

## Filter downloaded files from EPO portal

Choose files to split into Raw IDR files.

## Batch process downloaded EPO files into Raw IDRs

In [4]:
readpath = os.path.join(basepath, 'Downloads')
writepath = os.path.join(basepath, 'Raw_IDR')
error_log = []

for i, file in enumerate(output_df.files):
    try:
        os.chdir(readpath)
        filedf = pd.read_csv(file, sep = ",", header = 0)
        accts = ast.literal_eval(output_df.accts[i])
        IDRdrop.raw_split(filedf, readpath, writepath, output_df.util[i], accts)
        print('success, file: ', file)
        
    except:
        print('error, file: ', file)
        
print('RAW split complete.')

found 4 accounts.
success, file:  eversource_3301aae8-ccc5-444f-b3cc-59871f336398_b0946bc7_hourlycsv.csv
found 1 accounts.
success, file:  eversource_ea7fc589-6e46-4e50-9311-edcf240ef6eb_ca2a3e3d_hourlycsv.csv
found 4 accounts.
success, file:  eversource_6a4bf09e-05d0-4650-87d0-e5fc3a8cf588_f174016b_hourlycsv.csv
found 4 accounts.
success, file:  eversource_f77d133b-f6a9-4a30-bd74-6279ceb1b22e_1588f8ac_hourlycsv.csv
found 1 accounts.
success, file:  eversource_e8ccb470-8a69-4927-88ba-8b90a77c2507_6ae8ec3c_hourlycsv.csv
found 2 accounts.
success, file:  eversource_3cdbbc1e-68ba-48ea-8cbe-b4a77d15d30d_ad6531f4_hourlycsv.csv
found 1 accounts.
success, file:  eversource_368b06a1-eb32-4ab6-a29b-14d1282156c9_0b463bcc_hourlycsv.csv
RAW split complete.


## Show Raw IDR files based on utility

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

Choose Raw IDRs to filter into IDR files.

In [5]:
rawfiles = IDRdrop.show_dir(writepath, 25)
index = [36]

processfiles = list(rawfiles.files[:13])
print('files to process: ')
processfiles

files found in dir:  /Volumes/Samsung_T5/la-tools-test/IDR_Drop/Raw_IDR
files to process: 


['WMECO_54480402023_526541004_IDR_RAW.csv',
 'WMECO_54977381078_126733008_IDR_RAW.csv',
 'CLP_51436492052_684002003_IDR_RAW.csv',
 'CLP_51549893006_482562007_IDR_RAW.csv',
 'CLP_51874553092_860042003_IDR_RAW.csv',
 'CLP_51482323003_190712004_IDR_RAW.csv',
 'CLP_51640724076_352382007_IDR_RAW.csv',
 'CLP_51733934061_347092002_IDR_RAW.csv',
 'CLP_51801694019_298103007_IDR_RAW.csv',
 'CLP_51970204004_216762005_IDR_RAW.csv',
 'WMECO_54202385001_052931000_IDR_RAW.csv',
 'WMECO_54010781037_952531008_IDR_RAW.csv',
 'WMECO_54364315051_042703006_IDR_RAW.csv']

## Batch filter Raw IDR into IDR files to be dropped

In [7]:
readpath = os.path.join(basepath, 'Raw_IDR')
writepath = os.path.join(basepath, 'IDR')
error_log = []

for dropfile in processfiles:
    try:
        IDRdrop.data_drop(dropfile, readpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        error_log.append(dropfile)
        print("error, file: ", dropfile)

success, file:  WMECO_54480402023_526541004_IDR_RAW.csv
overlap removed at 2/6/2019, for file:
success, file:  WMECO_54977381078_126733008_IDR_RAW.csv
success, file:  CLP_51436492052_684002003_IDR_RAW.csv
overlap removed at 2/13/2019, for file:
success, file:  CLP_51549893006_482562007_IDR_RAW.csv
overlap removed at 6/27/2019, for file:
success, file:  CLP_51874553092_860042003_IDR_RAW.csv
success, file:  CLP_51482323003_190712004_IDR_RAW.csv
overlap removed at 8/10/2019, for file:
success, file:  CLP_51640724076_352382007_IDR_RAW.csv
success, file:  CLP_51733934061_347092002_IDR_RAW.csv
success, file:  CLP_51801694019_298103007_IDR_RAW.csv
success, file:  CLP_51970204004_216762005_IDR_RAW.csv
success, file:  WMECO_54202385001_052931000_IDR_RAW.csv
success, file:  WMECO_54010781037_952531008_IDR_RAW.csv
success, file:  WMECO_54364315051_042703006_IDR_RAW.csv


## Transform vertical data to horizontal

In [8]:
idr = IDRdrop.show_dir(writepath, 15)
idr

files found in dir:  /Volumes/Samsung_T5/la-tools-test/IDR_Drop/IDR


,files,time
0,CLP_51482323003_190712004_IDR.csv,2019-10-14 23:54:52
1,CLP_51874553092_860042003_IDR_3.csv,2019-10-14 23:54:52
2,WMECO_54202385001_052931000_IDR.csv,2019-10-14 23:54:52
3,CLP_51970204004_216762005_IDR.csv,2019-10-14 23:54:52
4,CLP_51801694019_298103007_IDR.csv,2019-10-14 23:54:52
5,CLP_51733934061_347092002_IDR.csv,2019-10-14 23:54:52
6,CLP_51640724076_352382007_IDR_3.csv,2019-10-14 23:54:52
7,CLP_51640724076_352382007_IDR.csv,2019-10-14 23:54:52
8,CLP_51874553092_860042003_IDR.csv,2019-10-14 23:54:52
9,WMECO_54010781037_952531008_IDR.csv,2019-10-14 23:54:52


In [9]:
for file in idr.files:
    IDRdrop.hor_to_vert(file)

transformed and wrote CLP_51482323003_190712004_IDR.csv to CLP_51482323003_190712004_IDR_vert.csv.
transformed and wrote CLP_51874553092_860042003_IDR_3.csv to CLP_51874553092_860042003_IDR_3_vert.csv.
transformed and wrote WMECO_54202385001_052931000_IDR.csv to WMECO_54202385001_052931000_IDR_vert.csv.
transformed and wrote CLP_51970204004_216762005_IDR.csv to CLP_51970204004_216762005_IDR_vert.csv.
transformed and wrote CLP_51801694019_298103007_IDR.csv to CLP_51801694019_298103007_IDR_vert.csv.
transformed and wrote CLP_51733934061_347092002_IDR.csv to CLP_51733934061_347092002_IDR_vert.csv.
transformed and wrote CLP_51640724076_352382007_IDR_3.csv to CLP_51640724076_352382007_IDR_3_vert.csv.
transformed and wrote CLP_51640724076_352382007_IDR.csv to CLP_51640724076_352382007_IDR_vert.csv.
transformed and wrote CLP_51874553092_860042003_IDR.csv to CLP_51874553092_860042003_IDR_vert.csv.
transformed and wrote WMECO_54010781037_952531008_IDR.csv to WMECO_54010781037_952531008_IDR_vert